In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

# Functions

If you need to do a computation once, you can run it step by step.  But almost always, you will want to run a computation many times.  In these cases, you should gather that code into a function.

In Python, functions are introduced with the `def` keyword:

In [ ]:
def subtract(a, b):
    diff = a - b
    return diff

The **arguments** to the function are placed in parentheses following the function name.  The value returned follows the `return` keyword.

Functions are called by appending their name with parentheses and the arguments.

In [ ]:
subtract(5, 3)

You can specify which argument is which with a `name=value` syntax.

In [ ]:
subtract(a=5, b=3)

When done this way, argument order doesn't matter.

In [ ]:
subtract(b=3, a=5)

Variables defined inside a function live within a function **context**, not the global namespace.

In [ ]:
'subtract' in globals(), 'a' in globals(), 'diff' in globals()

They can reference variables from the global scope, but any assignment will create a local variable.

In [ ]:
x = "Hello"

def global_var():
    print "In global_var", x

global_var()

def local_var():
    x = "World"
    print "In local_var", x

local_var()

print "Global namespace", x

If you want to modify a global variable, use the `global` keyword.

In [ ]:
def mod_global():
    global x
    x = "World"

mod_global()
print x

Python uses a "pass-by-object" paradigm.  This means that the same object is shared between the calling scope and the function scope.  The implications of this depend on the type of object being passed.

Immutable objects cannot ever be changed, so it should be unsurprising that they cannot be changed within a function call.

In [ ]:
a = 5
def func(x):
    x = x + 1
    return x

print func(a)
print a

When exectution of `func` starts, both `a` and `x` refer to the same object, the integer `5`.  But the line `x = x + 1` creates a new object, the integer `6`, and assigns that to `x`.  This does not affect the object `5`, which `a` still points to.

In contrast, mutable objects can be changed by functions.

In [ ]:
a = [5]
def func(x):
    x.append(1)
    return x

print func(a)
print a

This time, in `func` we modify the list to which `x` points.  Because `a` also points to this same list, it will show the modifications as well.

**Question:** Will this function modify its first argument, or create a new local variable?  What assumptions are you making?
```python
def func(x, y):
    x += y
```

## Functions as First-Class Objects

In Python, functions are **first-class objects**.  This means that they can be used just like any other data type.  As a result, there are sum functions that take other functions as arguments.

As a simple example, the `map` function will apply a function to each element in a list.  (You can do this more Pythonically with a list comprehension.)

In [ ]:
def add_one(x):
    return x + 1

map(add_one, range(5))

Sometimes you may not want to define a function only to use it as an argument to another function.  In these cases, anonymous functions can be handy.  These are introduced by the `lambda` keyword, and they have their arguments and return values separated by a colon.

In [ ]:
map(lambda x: x + 1, range(5))

Python has purposefully limited the power of lambdas.  They can contain only a single statement that yields a value.  For anything more complicated, you need to create a named function using the `def` keyword.  This is a Good Thing™, since it tends to make your code more readable.

Functions created with lambdas can be given names with variable assignment.  For most purposes, the following are equivalent.

In [ ]:
def add_one(x):
    return x + 1

add_one = lambda x: x + 1

## Closures

Functions can be defined in many places in Python, including within other functions.  And being first-class objects, they can be returned as values as well.  This means that you can write a function that returns a function, like so:

In [ ]:
def make_adder(n):
    def func(x):
        return x + n
    return func

plus_two = make_adder(2)

What is `plus_two`?

In [ ]:
type(plus_two)

Since it's a function, we can call it.

In [ ]:
plus_two(3)

That's all well and good, but you might wonder, "Where is `n` stored?"  It's not a global variable, but it's not defined anywhere within the function that became `plus_two`.

Instead, Python noticed that this function referred to a variable in an external scope.  As a result, it made a **closure** over that external variable.  This object will remain in memory as long as `plus_two` is in scope, even though there is no explicit reference to it.

You can actually access this value through some special properties of the function.

In [ ]:
plus_two.__closure__[0].cell_contents

## Variable Arguments and Keywords

Let's suppose we have a function that takes three arguments:

In [ ]:
def f(a, b, c):
    return a + b - c

f(1, 2, 3)

And suppose we want to call it with arguments that we have in a list.

In [ ]:
a = [1, 2, 3]

We can't call `f` directly with `a`, because `f` needs three arguments, not one list.

In [ ]:
try:
    f(a)
except TypeError as e:
    print e

We could index the arguments out of `a` manually, but there's a better way.  Python uses the star operator to do argument unpacking.  This turns the items in `a` into arguments of `f`.

In [ ]:
f(*a)

This works the other way in a function definition.  If you want your function to be able to take an arbitrary number of arguments, give it an argument preceeded by a star.

In [ ]:
def var_args(*args):
    print args

This will become a tuple with all the arguments.

In [ ]:
var_args(1, 2, 3, 'spam', None)

Of course, this can be combined with a function call using argument unpacking....

In [ ]:
var_args(*a)

The variable-length argument list can coexist with explicit arguments, but it must appear after them in the function definition.

In [ ]:
def many_args(a, b, *args):
    print a
    print b
    print args

In [ ]:
many_args(1, 2, 3, 'spam', None)

Similarly, arbitrary keyword arguments can be collected with the `**` operator.

In [ ]:
def kw_args(**kw):
    print kw

In [ ]:
kw_args(a=1, b=2, c=3)

And similarly, you can to keyword unpacking from a dictionary.

In [ ]:
f(**{'a': 1, 'b':2, 'c': 3})

Thus, a function that takes an arbitrary number of arguments and keywords could be defined as follows:

In [ ]:
def all_args(*args, **kw):
    print args
    print kw

In [ ]:
all_args(1, 2, a=3, b=4, c=5)

This explains how the dictionary constructor works, for example.

In [ ]:
dict(a=1, b=2)

One common use for these techniques is for creating wrappers for other functions.  Suppose we have some complicated function that takes a bunch of keyword arguments.  They are given default values, so usually you don't actually need to specify them.

In [ ]:
def complicated(a, b=1, c=None, d=0, e='default'):
    """This is a complicated function with several optional arguments."""
    print "Doing something complicated", a, b, c, d, e

Now, suppose we wanted to create a wrapper that would call this function.  We want to be able to specify any of those keyword arguments in the wrapper and have them passed to the the function.  This would be a lot of work to do explicitly, and it would require us to update the wrapper each time the signature of the underlying function changed.

Instead, we can just accept a `**kw` argument and pass all of those on to the underlying function.

In [ ]:
def complicated_wrapper(a, **kw):
    print "About to do something complicated"
    complicated(a + 1, **kw)

In [ ]:
complicated_wrapper(5, d=17, e='other')

The downside is that we don't get any introspection help on the acceptable keyword arguments or docstring.

In [ ]:
help(complicated_wrapper)

## Decorators

Recall the `subtract` function we defined at the beginning of the notebook.  Suppose we wanted to repeat that operation, but ensure that the result was non-negative.  How would we go about that?

A simple approach would be to create a wrapper, as we did above.

In [ ]:
def nonneg_sub(*args):
    v = subtract(*args)
    return v if v >=0 else 0

print subtract(2, 5)
print nonneg_sub(2, 5)

This works fine, but it doesn't scale.  If we want to wrap a bunch of other functions (say, `add`, `multiply`, `divide`, etc.) we'd have to define a wrapper for each.  Instead, let's write a function to do that work.

This function will take a function as an argument and return another function.  Here's an example:

In [ ]:
def ensure_nonneg(func):
    def wrapper(*args):
        v = func(*args)
        return v if v >= 0 else 0
    return wrapper

nonneg_sub2 = ensure_nonneg(subtract)
nonneg_sub2(5, 2), nonneg_sub2(2, 5)

Let's take that step by step:
1. `ensure_nonneg` takes `subtract` as an argument.
2. `ensure_nonneg` defines a function `wrapper`.
3. `wrapper` is a closure, which keeps a reference to `subtract`.
4. `ensure_nonneg` returns the wrapper, which we assign the name `nonneg_sub2`.
5. When we call `nonneg_sub2`, we're calling the code defined in `wrapper`, which in turn calls `subtract`.
6. The wrapper code checks the sign of the return value of `subtract`, returning it or 0, whichever is greater.

Sometimes, this wrapping is so important that we don't want to keep the original name around -- we only want to use the wrapped version.  In that case, we could assign the wrapped copy back to the original name of the function.

In [ ]:
print subtract(2, 5)
subtract = ensure_nonneg(subtract)
print subtract(2, 5)

A decorator in Python just does the wrapping automatically:

In [ ]:
@ensure_nonneg
def subtract(a, b):
    return a - b

This defines the function `subtract`, immediately passes it as an argument t `ensure_nonneg`, and the assigns the output of `ensure_nonneg` back to the name `subtract`.

In [ ]:
subtract(5, 2), subtract(2, 5)

Decorators can be used for a variety of purposes, including logging (write to a file each time a function is called), diagnostics (start and stop a timer before and after each function call, to see how long it takes), and caching results (save the output of a function, and return that value immediately when it is called again).

Sometimes you want to have a whole family of decorators.  In these cases, we can define a function that returns a decorator.  For example, if we want to ensure a function's return value will be at least a certain value:

In [ ]:
def ensure_atleast(n):
    def decorator(func):
        def wrapper(*args):
            v = func(*args)
            return v if v >= n else n
        return wrapper
    return decorator

Then we can use it like so:

In [ ]:
@ensure_atleast(10)
def subtract(a, b):
    return a - b

subtract(5, 2)

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*